In [ ]:
import folium
import io
from PIL import Image
import folium.features
import json
import requests
from folium.plugins import FloatImage
import base64
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime



In [60]:
#Hente ned informasjon om en vegstegning

def hentVegmelding(simpleDetailsUrl):
    #simpleDetailsUrl = feat["properties"]["roadFeatures"]["trafficMessages"][0]['simpleDetailsUrl']
    headers = {"Accept":"application/vnd.svv.v1+json;charset=utf-8","X-System-ID": "RouteMonitor - ITS seksjonen"}
    trafficMessageReq = requests.get(simpleDetailsUrl,headers=headers)
    print(simpleDetailsUrl)
    trafficMessage = trafficMessageReq.json()
    print(trafficMessage)

In [62]:

# Oppsett av standard verdier for kalulasjoner
#
showPopup = True # vis popup med info om stengte veier når kartet åpnes
makeImage = True # lag bilde av kartet
makeWeb = True # Lager en hjemmeside av resultatet
height = 0 # høyde på kjøretøy, i meter
bruksklasse = "Bk_10_50" # mulige bruksklasser (sett til "" for vanlig bil): [ Bk_6_28, Bk_8_32, Bk_T8_40, Bk_T8_50, Bk_10_42, Bk_10_50, Bk_10_56, Bk_10_60 ]

baseurl = "https://www.vegvesen.no/ws/no/vegvesen/ruteplan/routingservice_v3_0/open/routingservice/api/Route/bruksklasseNormalTransport"

Trondheim = [63.42839, 10.48198]
Brekstad = [63.69380, 9.66100]
Bodo = [67.28785, 14.39415]
Oslo = [59.90813, 10.74873]

stops = f"{Trondheim[1]},{Trondheim[0]};{Oslo[1]},{Oslo[0]}"
print(stops)

payload = {'stops': stops, 
           'bruksklasse': bruksklasse,
           'Height': height,
           'InputSRS': 'EPSG_4326',
           'OutputSRS': 'EPSG_4326',
           'ReturnFields':'Geometry' }

print(payload)

r = requests.get(baseurl, params=payload)

#Sjekke return
print("Status for API request")
print(r.status_code)
print(r.reason)

data = r.json()

numberOfRoutesFound = len(data['routes'])
print(numberOfRoutesFound)



# Antar at første rute er beste rute før alle kadidatruter sjekkes
# Flytter flytter viktig informasjon inn i properties for å få ekse geojson
besteRuteIndex = 0
besteRuteVerdi = data['routes'][0]['statistic']['totalDriveTime']
for i in range(0,numberOfRoutesFound): #begynner på 1 fordi vi antar at første rute er beste og bare sjekker resten
    i = int(i)
    # Legger rutenavn til geojson objekt på rett måte
    antallFestures= len(data['routes'][i]['features'])
    
    for j in range(0,antallFestures):
        data['routes'][i]['features'][j]['properties']['routeName'] = data['routes'][i]['routeName']
        # Hente inn alle trafikkmeldinger
        if data['routes'][i]['features'][j]['properties']['roadFeatures']['trafficMessages']:
            trafikkMeldingsListe = []
            for tafikkmelding in data['routes'][i]['features'][j]['properties']["roadFeatures"]["trafficMessages"]:
                trafikkMeldingsListe.append(tafikkmelding['type'])
                if tafikkmelding['simpleDetailsUrl']:
                    hentVegmelding(tafikkmelding['simpleDetailsUrl'])
                    
                trafikkMeldingsListe.append(str(tafikkmelding))
                
                #if tafikkmelding['type']
            data['routes'][i]['features'][j]['properties']['trafficMessages'] =  trafikkMeldingsListe
        else:
            data['routes'][i]['features'][j]['properties']['trafficMessages'] =  ['']
        
    
    if data['routes'][i]['statistic']['totalDriveTime'] < besteRuteVerdi:
        besteRuteIndex = i
        besteRuteVerdi = data['routes'][i]['statistic']['totalDriveTime']
        
        
print(f"Beste rute har index: {besteRuteIndex} og heter: {data['routes'][besteRuteIndex]['routeName']} og har verdi: {besteRuteVerdi}")




besteRuteGeoJson = {
            "type": "FeatureCollection",
            "features": data['routes'][besteRuteIndex]['features']
        }

#print(dataOmRuta)

#tmp = data['routes'][0]


10.48198,63.42839;10.74873,59.90813
{'stops': '10.48198,63.42839;10.74873,59.90813', 'bruksklasse': 'Bk_10_50', 'Height': 0, 'InputSRS': 'EPSG_4326', 'OutputSRS': 'EPSG_4326', 'ReturnFields': 'Geometry'}
Status for API request
200
OK
2
https://traffic-info.atlas.vegvesen.no/traffic-information/messages/NPRA_HBT_17-02-2025.34721
{'metadata': {'overallDataStatus': 'ok', 'pagination': {'totalElements': 1, 'matchedElements': 0}}, 'trafficMessages': [{'dataFetchStatus': 'ok', 'publicationTime': '2025-03-08T23:48:07+01:00', 'id': 'NPRA_HBT_17-02-2025.34721', 'type': 'RoadWork', 'icon': {'mapMarkerIcon': 'RoadWork', 'position': {'type': 'Point', 'coordinates': [10.30497, 63.218704]}}, 'descriptionOfLocation': 'E6 Nyhus i Melhus, Trøndelag - E6 Øyan i Melhus, Trøndelag', 'descriptionOfTrafficMessage': 'Vegarbeid.', 'isActiveNow': True, 'isAtNight': True, 'isDayTime': True, 'trafficImpact': 'none', 'startTime': '2025-02-17T00:00:00+01:00', 'estimatedEndTime': '2025-04-11T23:59:59+02:00', 'updat

In [58]:
map = folium.Map([69.1, 15.8],zoom_start=10,zoom_control=False)
g = folium.GeoJson(besteRuteGeoJson,
                style_function=lambda feature: {
                    "color": "red",
                    "weight": 2,
                    "dashArray": "5, 5",
                },
                tooltip=folium.features.GeoJsonTooltip(
            fields=['routeName', 'trafficMessages'],
            aliases=['Rutenavn', 'Trafikkmeldinger'])
            ).add_to(map)
folium.FitOverlays(max_zoom=12).add_to(map)
map